In [1]:
#MODULOS DE SELENIUM
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

#MODULOS SECUNDARIOS
import pandas as pd
import numpy as np
import os
import re
import time
import ssl
import sys
import math
import traceback


#MODULO ALERTAS DESACTIVADAS
import warnings
warnings.filterwarnings('ignore')

#MODULO PROPIOS
from funciones_primarias import validacion
from funciones_secundarias import barra_carga,scroll_down

In [2]:
'''Protocoloes SSL'''
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [3]:

'''
OBJETO DRIVER 
    - Opciones 
    - URL
'''

#Opciones Chrome
chrome_options = Options()
chrome_options.add_experimental_option("prefs", {
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
  })

#Instancia driver -  Acceso a la URL

INSTAGRAM_URL = 'https://www.instagram.com/reel/Cv-e8k2pvaq/?igshid=MTc4MmM1YmI2Ng%3D%3D'
PAGINA_URL    = 'https://www.instagram.com/'

driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))
driver.get(PAGINA_URL)

time.sleep(5)



In [4]:
'''ACCESO A LA CUENTA'''

# Encuentra los campos de entrada para el nombre de usuario y la contraseña
username_input = driver.find_element(By.XPATH, value = "//input[@name='username']")
password_input = driver.find_element(By.XPATH, value = "//input[@name='password']")

# Ingresa tus credenciales de inicio de sesión
username_input.send_keys("j.alonsoordinola@gmail.com")
password_input.send_keys("Libertad5%")

# Envía el formulario de inicio de sesión
password_input.send_keys(Keys.RETURN)

# Espera un momento para el inicio de sesión
barra_carga(10)
print('Acceso Exitoso')


12345678910-------------]
Acceso Exitoso


In [5]:
'''ACCESO A EL POST AL CUAL SE LE EXTRAERAN LOS COMENTARIOS'''

driver.get(INSTAGRAM_URL)
driver.maximize_window()
barra_carga(10)
print('Post Cargado')

12345678910-------------]
Post Cargado


In [6]:
# '''Prueba de click en la pantalla/DELL 15'''
# actions = ActionChains(driver)

# # Mueve el mouse a una posición específica relativa al punto actual del mouse
# # En este ejemplo, moveremos el mouse 100 píxeles a la derecha y 50 píxeles hacia abajo
# actions.move_by_offset(650, 690)

# # Realiza un clic derecho en la ubicación actual del mouse
# actions.context_click()

# # Ejecuta las acciones
# actions.perform()

# # # Usa JavaScript para obtener las coordenadas del mouse
# # x_coord = driver.execute_script("return window.scrollX + arguments[0];", 100)
# # y_coord = driver.execute_script("return window.scrollY + arguments[0];", 50)

# # # Imprime las coordenadas del mouse
# # print(f"Coordenadas del mouse - X: {x_coord}, Y: {y_coord}")

In [7]:
''' 
VALIDACION 

Funcionamiento de acceso a la URL, navegacion y captura de segmentos del en X.Path
Prueba del Scroll y de la Estrategia de Espera para la carga de mensajes

'''

#ubicar el primer comentario que se encuentra en la pagina, es decir el primer
#contenedor dentro de contenedor macro, esto para tener el punto de referencia de :
#donde se encuentra el bloque de comentarios y donde inicia

seccion_comments = '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/section/main/div/div[1]/div/div[2]/div/div[2]/div/div/div['

try:
    driver.find_element(By.XPATH, value = seccion_comments +'1]')
    print('SeccionComentario Encontrada',driver.find_element(By.XPATH, value = seccion_comments +'1]'))
except: print('nada')

barra_carga(20)

SeccionComentario Encontrada <selenium.webdriver.remote.webelement.WebElement (session="8a9f6f102e3297deba6c6628606cd95d", element="5DD6C66C8EAE9AC670FA2339C032B43B_element_150")>
1234567891011121314151617181920-------------]


In [ ]:
#VARIABLES PARA EL SCRAPING
comentarios_recorridos = list()
centinela = True
comment_externo = 1
comment_interno = 1
base_final = pd.DataFrame()


while centinela == True:
    try:
        
        try:
            driver.find_element(By.XPATH, value= '/html/body/div[2]/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/section/main/div/div[1]/div/div[2]/div/div[2]/div/div/div[1]/div/div[2]/div/span/div/div/span[1]/div/a')
            print('Nombre de Usuario')
            time.sleep(30)
        except:
            pass

        # Level 1 comment principal
        level = 1
        # Detectar el DISPLAY NAME del usuario
        try:
            display_name = driver.find_element(By.XPATH, value = seccion_comments + str(comment_externo) + "]/ytd-comment-renderer/div[3]/div[2]/div[1]/div[2]/h3/a/span").text
            print(display_name)
        except:
            centinela = False
            break

        try:    
        # Detectar NOMBRE DE USUARIO (toma como punto de referencia en Display Name del bloque anterior)
            username_thread = driver.find_element(by=By.PARTIAL_LINK_TEXT,value=display_name).get_attribute("href")
        # Detectar comment principal
            comment = driver.find_element(By.XPATH, value = seccion_comments + str(comment_externo) + "]/ytd-comment-renderer/div[3]/div[2]/div[2]/ytd-expander/div/yt-formatted-string").text
    #         # Detectar fecha
    #         fecha = driver.find_element(By.XPATH, value = seccion_comments + str(comment_externo) + "]/div[1]/div[1]/p[2]/span[1]").text
    #         # Detectar número de likes
    #         likes = driver.find_element(By.XPATH, value = seccion_comments + str(comment_externo) + "]/div[1]/div[1]/p[2]/div/span").text
            print(str(comment))

        except:
            username_thread = driver.find_element(by=By.PARTIAL_LINK_TEXT,value=display_name).get_attribute("href")
            # Detectar comment principal
            comment = driver.find_element(By.XPATH, value = seccion_comments + str(comment_externo) + "]/ytd-comment-renderer/div[3]/div[2]/div[2]/ytd-expander/div/yt-formatted-string").text
            print( 'Extraccion dos'+ str(comment))    

        # Creacion de Dataframe con los datos de cada comentario, y la base previa 
        d = {"level": level, "display_name": display_name, "username":username_thread, "comment":comment}
        base_temporal = pd.DataFrame(data=d,index=[comment_externo])
        base_final = pd.concat([base_final, base_temporal], ignore_index=True)
    
        # Actualizacion del loop para continuar con el recorrido
        comentarios_recorridos.append(comment_externo)
        comment_externo = comment_externo + 1
        print("Acabé el comment " + str(comment_externo - 1) + ' perteneciente al usuario: ' + str(display_name))
        
        x=0
        driver.execute_script('scrollBy(0,150)')
        time.sleep(5)
        
    except Exception as e:
        traceback.print_exc()
        time.sleep(50)
        scroll_down(3, 1, 850,driver)
        # with open(r'C:\Users\JOSE\Desktop\Trabajo\BX\Tiktok\progreso.txt', 'w') as fp:
        #     fp.write(str(comentarios_recorridos[-1]))
        print(f'Error {type(e)}: e')
        # base_final.to_excel('Base_tiktok_medicossinfronteras.xlsx')
        
print('CARGA COMPLETA')

driver.quit()

base_final.to_excel('base_youtube_cuchicheo_05.xlsx')   

In [ ]:
cuchi01 = pd.read_excel('base_youtube_cuchicheo_01.xlsx')
cuchi02 = pd.read_excel('base_youtube_cuchicheo_02.xlsx')
cuchi03 = pd.read_excel('base_youtube_cuchicheo_03.xlsx')
cuchi04 = pd.read_excel('base_youtube_cuchicheo_04.xlsx')
cuchi05 = pd.read_excel('base_youtube_cuchicheo_05.xlsx')

base_consolidada_youtube_cuchicheo = pd.concat([cuchi01,cuchi02,cuchi03,cuchi03,cuchi04,cuchi05])
base_consolidada_youtube_cuchicheo.to_excel('base_consolidada_youtube_cuchicheo.xlsx',index=False)